# GPT-SoVITS on Sagemaker

## build image

In [1]:
!chmod +x ./*.sh && ./build_and_push.sh 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile-sagemaker             0.0s
 => => transferring dockerfile: 1.70kB                                     0.0s
 => [internal] load metadata for docker.io/cnstark/pytorch:2.0.1-py3.9.17  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile-sagemaker             0.0s
 => => transferring dockerfile: 1.70kB                                     0.0s
 => [internal] load metadata for docker.io/cn

In [2]:
!pip install sagemaker-ssh-helper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
## SM endpoint test

In [ ]:
###create sagemaker model

In [1]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
image="gpt-sovits-inference"
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

full_image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
print(full_image_uri)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/08/25 08:31:21] INFO     Found credentials from IAM Role:                                   ]8;id=579307;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=706229;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[02/08/25 08:31:28] INFO     Found credentials from IAM Role:                                   ]8;id=336410;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=527727;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=168655;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=310262;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[02/08/25 08:31:29] INFO     Found credentials from IAM Role:                                   ]8;id=989858;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=312545;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

134927008626.dkr.ecr.us-west-2.amazonaws.com/gpt-sovits-inference:latest


In [2]:
from time import gmtime, strftime
## for debug only
sm_client = boto3.client(service_name='sagemaker')

def create_model():
    image=full_image_uri
    model_name="gpt-sovits-sagemaker-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        ExecutionRoleArn=role,
        Containers=[{"Image": image}],
    )
    print(create_model_response)
    return model_name

In [3]:
model_name=create_model()

{'ModelArn': 'arn:aws:sagemaker:us-west-2:134927008626:model/gpt-sovits-sagemaker-2025-02-08-08-32-15', 'ResponseMetadata': {'RequestId': '52bc47df-b607-4549-aa33-4f62573322fd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '52bc47df-b607-4549-aa33-4f62573322fd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '102', 'date': 'Sat, 08 Feb 2025 08:32:16 GMT'}, 'RetryAttempts': 0}}


In [ ]:
###create endpoint configuration¶

In [4]:
endpointConfigName = "gpt-sovits-sagemaker-configuration-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint_configuration():
    create_endpoint_config_response = sm_client.create_endpoint_config(     
        EndpointConfigName=endpointConfigName,
        ProductionVariants=[
            {
                #"ModelName":"gpt-sovits-sagemaker-012024-03-28-04-00-03",
                "ModelName":model_name,
                "VariantName": "gpt-sovits-sagemaker"+"-variant",
                "InstanceType": "ml.g5.xlarge",  # 指定 g5.xlarge 机器
                "InitialInstanceCount": 1,
                "ModelDataDownloadTimeoutInSeconds": 1200,
                "ContainerStartupHealthCheckTimeoutInSeconds": 1200
            }
        ],
    )
    print(create_endpoint_config_response)
    return endpointConfigName


In [5]:
create_endpoint_configuration()

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:134927008626:endpoint-config/gpt-sovits-sagemaker-configuration-2025-02-08-08-32-19', 'ResponseMetadata': {'RequestId': '3a28edfe-3827-40e8-9f48-8a885fc0fb36', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3a28edfe-3827-40e8-9f48-8a885fc0fb36', 'content-type': 'application/x-amz-json-1.1', 'content-length': '135', 'date': 'Sat, 08 Feb 2025 08:32:24 GMT'}, 'RetryAttempts': 0}}


'gpt-sovits-sagemaker-configuration-2025-02-08-08-32-19'

In [ ]:
##create endpoint

In [6]:
endpointName="gpt-sovits-sagemaker-endpoint"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint():
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpointName,
        #EndpointConfigName="gpt-sovits-sagemaker-configuration2024-03-28-04-03-53",
        EndpointConfigName=endpointConfigName
    )
    print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    resp = sm_client.describe_endpoint(EndpointName=endpointName)
    print("Endpoint Status: " + resp["EndpointStatus"])
    print("Waiting for {} endpoint to be in service".format("gpt-sovits-sagemaker-endpoint"))
    waiter = sm_client.get_waiter("endpoint_in_service")
    waiter.wait(EndpointName=endpointName)

In [7]:
create_endpoint()

Endpoint Arn: arn:aws:sagemaker:us-west-2:134927008626:endpoint/gpt-sovits-sagemaker-endpoint2025-02-08-08-32-29
Endpoint Status: Creating
Waiting for gpt-sovits-sagemaker-endpoint endpoint to be in service


## Realtime inferecne with sagemaker endpoint

## Streams test (only for stream branch deployment)

In [12]:
import wave
import json
from collections import defaultdict

endpointName = "gpt-sovits-sagemaker-endpoint2025-02-08-08-32-29"

def invoke_streams_endpoint(smr_client, endpointName, request):
    content_type = "application/json"
    payload = json.dumps(request, ensure_ascii=False)
    response_model = smr_client.invoke_endpoint_with_response_stream(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )
    print(response_model['ResponseMetadata'])
    event_stream = response_model['Body']
    
    result = defaultdict(dict)
    for index, event in enumerate(event_stream):
        chunk = event['PayloadPart']['Bytes']
        
        result[index] = {
            'first_chunk': index == 0,
            'bytes': chunk,
            'last_chunk': False,
            'index': index
        }
        
        if index < 5:
            print(f"chunk {index} len:", len(chunk))
    
    return list(result.values())

def audio_chunks_to_wav(audio_chunks, output_filename, channels=1, sample_width=2, sample_rate=16000):
    with wave.open(output_filename, 'wb') as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(sample_rate)
        for chunk in audio_chunks:
            wav_file.writeframes(chunk['bytes'])
    print(f"WAV file '{output_filename}' has been created.")

In [13]:
# Copy reference wav into S3
! aws s3 cp ./ref-4.wav s3://$bucket/gpt-sovits/wav_ref/ref-4.wav

upload: ./ref-4.wav to s3://sagemaker-us-west-2-134927008626/gpt-sovits/wav_ref/ref-4.wav


In [14]:
runtime_sm_client = boto3.client(service_name="sagemaker-runtime", region_name=region)

request = {"refer_wav_path": f"s3://{bucket}/gpt-sovits/wav_ref/ref-4.wav",
    "prompt_text": "脑瘤的发病率，可以说是，这几年越来越高",
    "prompt_language":"zh",
    "text":"脑肿瘤是指脑组织中形成肿瘤的细胞异常生长。肿瘤可以原发于颅内脑组织，也可以由身体其他部位恶性肿瘤转移而来。原发性脑肿瘤可以是良性（非癌性）或恶性的。脑肿瘤通常通过 CT（计算机断层扫描）及磁共振成像等多种检查来进行诊断。",
    "text_language" :"zh",
    "output_s3uri":"",
    "cut_punc":",，.。:：!！\"”'‘"
}


In [15]:
response=invoke_streams_endpoint(runtime_sm_client,endpointName,request)

{'RequestId': '0785208a-fb6a-47eb-bf9e-9daefc395b9f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0785208a-fb6a-47eb-bf9e-9daefc395b9f', 'x-amzn-invoked-production-variant': 'gpt-sovits-sagemaker-variant', 'x-amzn-sagemaker-content-type': 'application/json', 'date': 'Sat, 08 Feb 2025 08:42:18 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive'}, 'RetryAttempts': 0}
chunk 0 len: 139
chunk 1 len: 208
chunk 2 len: 272
chunk 3 len: 21
chunk 4 len: 308


In [16]:
audio_chunks_to_wav(response, "output-3-new.wav", sample_rate=32000)

WAV file 'output-3-new.wav' has been created.
